<a href="https://colab.research.google.com/github/PiyushWaradkar/LLM_Shopping_Service/blob/main/LLM_Shopping_Service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

In [ ]:
!pip install pandas openpyxl requests faiss-cpu

In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import time
import faiss
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import re

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
OLLAMA_URL = "https://554b-208-59-146-203.ngrok-free.app" #Invalid till you launch your own



# Verify LLM Run

In [ ]:
def verify_ollama():
    try:
        data = {"prompt": "Test", "model": "llama3", "stream": False}
        resp = requests.post(f"{OLLAMA_URL}/api/generate", json=data, timeout=10)
        resp.raise_for_status()
        print("Ollama server is accessible!")
        return True
    except Exception as e:
        print("Error connecting to Ollama server:", e)
        return False

def ollama_generate(prompt, model="llama3", system=None, stream=False):
    data = {"prompt": prompt, "model": model, "stream": stream}
    if system:
        data["system"] = system
    resp = requests.post(f"{OLLAMA_URL}/api/generate", json=data, timeout=300)
    if resp.status_code != 200:
        raise RuntimeError(f"Ollama error: {resp.status_code} - {resp.text}")
    return resp.text

In [ ]:
def load_data():
    doc_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
    products = pd.read_csv(
        f"https://docs.google.com/spreadsheets/d/{doc_id}/gviz/tq?tqx=out:csv&sheet=products"
    )
    emails = pd.read_csv(
        f"https://docs.google.com/spreadsheets/d/{doc_id}/gviz/tq?tqx=out:csv&sheet=emails"
    )
    print("Data loaded successfully\n")
    print("Products Preview:")
    display(products.head(2))
    print("\nEmails Preview:")
    display(emails.head(2))
    return products, emails

# Classify emails

In [ ]:
class EmailClassifier:
    def __init__(self, model="llama3"):
        self.model = model
        self.retries = 2
        self.keywords = {
            'order_request': [
                r'\b(order|buy|purchase)\b.*\d+',
                r'\b(send me|want|need)\b.*\d+',
                r'\b(quantity of|quantity:\s*)\d+'
            ],
            'product_inquiry': [
                r'\b(question about|details on)\b',
                r'\b(how|what|which|does|is)\b.*\?',
                r'\b(recommend|suitable|advice)\b'
            ]
        }

    def classify_all(self, emails_df):
        emails_df['category'] = emails_df.progress_apply(self._classify_email, axis=1)
        return emails_df

    def _classify_email(self, row):
        email_id = row['email_id']
        message = str(row['message'])

        llm_result = self._structured_llm_classification(email_id, message)
        if llm_result: return llm_result

        pattern_result = self._regex_classification(message)
        if pattern_result: return pattern_result

        return self._keyword_fallback(message)

    def _structured_llm_classification(self, email_id, message):
      prompt = f"""Return ONLY valid JSON without any extra text:
  {{
      "email_id": "{email_id}",
      "category": "order_request" or "product_inquiry",
      "reason": "1-sentence explanation"
  }}

  **Rules:**
  - "order_request" requires product/quantity mentions
  - "product_inquiry" requires questions about products

  **Email Content:**
  {message[:600]}"""

      for attempt in range(self.retries):
          try:
              response = ollama_generate(
                  prompt,
                  model=self.model,
                  system="You are a classification bot. Only return valid JSON.",
              )

              json_str = response.strip()
              data = json.loads(json_str)

              if data.get('category', '').lower() in ['order_request', 'product_inquiry']:
                  return data['category'].lower()

          except Exception as e:
              print(f"Retry {attempt+1} for {email_id}: {str(e)}")
              time.sleep(1.5)
      return None


    def _regex_classification(self, message):
        content = message.lower()

        for pattern in self.keywords['order_request']:
            if re.search(pattern, content, re.IGNORECASE):
                return "order_request"

        for pattern in self.keywords['product_inquiry']:
            if re.search(pattern, content, re.IGNORECASE):
                return "product_inquiry"

        return None

    def _keyword_fallback(self, message):
        content = message.lower()
        order_words = {'cart', 'checkout', 'ship', 'stock', 'inventory'}
        inquiry_words = {'compare', 'difference', 'recommend', 'durable'}

        if any(word in content for word in order_words):
            return "order_request"
        if any(word in content for word in inquiry_words):
            return "product_inquiry"

        return "unclassified"

# Process order requests

In [ ]:
class OrderManager:
    def __init__(self, products_df, model="llama3"):
        self.model = model
        self.products = products_df.copy().reset_index(drop=True)

        texts = (self.products["name"] + " " +
                 self.products["category"] + " " +
                 self.products["description"]).tolist()
        emb = [self._naive_embed(t) for t in texts]
        dim = len(emb[0])
        self.index = faiss.IndexFlatL2(dim)
        self.index.add(np.array(emb, dtype=np.float32))

    def _naive_embed(self, text):
        val = float(sum(ord(c) for c in text) % 10000)
        return [val]

    def _extract_order_details(self, message):
        prompt = (
            "Extract product orders from this email.\n"
            'Return JSON in the format: {"products":[{"product_id":"X","quantity":2},...]}\n\n'
            f"Email:\n{message}\n"
        )
        raw = ollama_generate(prompt, model=self.model)
        try:
            details = json.loads(raw)

            if details.get("products"):
                return details
            else:
                raise ValueError("Empty products in extracted JSON.")
        except Exception as e:
            print("LLM extraction failed, falling back to regex extraction. Error:", e)
            print("Raw LLM output was:", raw)

            found_products = []

            for pid in self.products['product_id'].unique():
                if pid.lower() in message.lower():
                    pattern = rf'(\d+)\s*(?:pair[s]?|pcs|units)?\s*.*{re.escape(pid)}'
                    m = re.search(pattern, message, flags=re.IGNORECASE)
                    qty = int(m.group(1)) if m else 1
                    found_products.append({"product_id": pid, "quantity": qty})
            return {"products": found_products}

    def _suggest_alternatives(self, pid, k=3):
        row = self.products[self.products.product_id == pid]
        if row.empty:
            return pd.DataFrame()
        txt = row.iloc[0]['name'] + " " + row.iloc[0]['category'] + " " + row.iloc[0]['description']
        v = np.array([self._naive_embed(txt)], dtype=np.float32)
        _, idx = self.index.search(v, k)
        results = []
        for i in idx[0]:
            results.append(self.products.iloc[i])
        return pd.DataFrame(results)

    def process_order(self, message, email_id):
        details = self._extract_order_details(message)
        results = []
        for item in details.get("products", []):
            pid = item.get("product_id", "")
            qty = item.get("quantity", 1)
            pid = re.sub(r'\s+', '', pid).upper()
            match = self.products[self.products.product_id == pid]
            if match.empty:
                results.append({
                    "email_id": email_id,
                    "product_id": pid,
                    "quantity": qty,
                    "status": "invalid_product",
                    "alternatives": None
                })
                continue
            stock_now = match.iloc[0]["stock"]
            if stock_now >= qty:
                idx = match.index[0]
                self.products.at[idx, "stock"] = stock_now - qty
                results.append({
                    "email_id": email_id,
                    "product_id": pid,
                    "quantity": qty,
                    "status": "created",
                    "alternatives": None
                })
            else:
                alts = self._suggest_alternatives(pid).to_dict("records")
                results.append({
                    "email_id": email_id,
                    "product_id": pid,
                    "quantity": qty,
                    "status": "out_of_stock",
                    "alternatives": alts
                })

        print(f"Processed order for {email_id}: {results}")
        return results

    def generate_order_response(self, grouped):
        responses = []
        for eid, items in grouped.items():
            resp_text = self._gen_single_response(eid, items)
            responses.append({"email_id": eid, "response": resp_text})
        return responses

    def _gen_single_response(self, email_id, items):
        summary = ""
        for it in items:
            summary += f"PID: {it['product_id']}, Q: {it['quantity']}, Status: {it['status']}.\n"
            if it["status"] == "out_of_stock" and it["alternatives"]:
                summary += "Alternatives:\n"
                for alt in it["alternatives"]:
                    summary += f"- {alt['product_id']} (stock: {alt['stock']})\n"
        prompt = (
            f"Order request from email {email_id}:\n{summary}\n"
            "Write a concise, professional response.\n"
            "If an item is 'created', confirm the order; if 'out_of_stock', mention the items and suggest alternatives.\n"
            "Return only the response text."
        )
        return ollama_generate(prompt, model=self.model)

# Handle product inquiry

In [ ]:
class ProductInquiryHandler:
    def __init__(self, products_df, model="llama3"):
        self.model = model
        self.products = products_df.copy().reset_index(drop=True)
        texts = (
            self.products['name'] + " " +
            self.products['category'] + " " +
            self.products['description']
        ).tolist()
        self.index, self.mapping = self._build_index(texts)

    def _build_index(self, texts):
        mapping = []
        vectors = []
        for i, t in enumerate(texts):
            val = float(sum(ord(c) for c in t) % 10000)
            vectors.append([val])
            mapping.append(i)
        idx = faiss.IndexFlatL2(1)
        idx.add(np.array(vectors, dtype=np.float32))
        return idx, mapping

    def handle_inquiry(self, message, k=3):
        if self._detect_language(message) != 'en':
            message = self._translate_to_english(message)
        val = float(sum(ord(c) for c in message) % 10000)
        v = np.array([[val]], dtype=np.float32)
        _, idx = self.index.search(v, k)
        relevant = []
        for i in idx[0]:
            relevant.append(self.products.iloc[self.mapping[i]])
        context_str = ""
        for row in relevant:
            context_str += f"ID:{row['product_id']}, Name:{row['name']}, Stock:{row['stock']}\nDesc:{row['description']}\n\n"
        prompt = (
            f"Customer inquiry:\n{message}\n\n"
            f"Relevant product info:\n{context_str}\n"
            "Write a concise, friendly reply referencing product IDs if needed.\n"
            "Return only the response text."
        )
        return ollama_generate(prompt, model=self.model)

    def _detect_language(self, text):
        prompt = f"Detect language of this text (respond ONLY with language code):\n{text[:300]}"
        return ollama_generate(prompt, model=self.model).strip()

    def _translate_to_english(self, text):
        prompt = f"Translate to English (respond ONLY with translation):\n{text[:500]}"
        return ollama_generate(prompt, model=self.model)

# Generate Results

In [ ]:
def generate_spreadsheet(emails_df, order_lines, order_responses, inquiry_responses, updated_products):
    wb = Workbook()

    ws1 = wb.active
    ws1.title = "email-classification"
    df_class = emails_df[["email_id", "category"]].copy()
    for r in dataframe_to_rows(df_class.rename(columns={"email_id": "email ID"}), index=False, header=True):
        ws1.append(r)

    ws2 = wb.create_sheet("order-status")
    df_order_lines = pd.DataFrame(order_lines)
    if df_order_lines.empty:
        df_order_lines = pd.DataFrame(columns=["email_id", "product_id", "quantity", "status"])
    df_order_lines = df_order_lines.rename(columns={"email_id": "email ID", "product_id": "product ID"})
    rows2 = dataframe_to_rows(df_order_lines[["email ID", "product ID", "quantity", "status"]], index=False, header=True)
    for r in rows2:
        ws2.append(r)

    ws3 = wb.create_sheet("order-response")
    df_ord_resp = pd.DataFrame(order_responses)
    if df_ord_resp.empty:
        df_ord_resp = pd.DataFrame(columns=["email_id", "response"])
    df_ord_resp = df_ord_resp.rename(columns={"email_id": "email ID"})
    rows3 = dataframe_to_rows(df_ord_resp[["email ID", "response"]], index=False, header=True)
    for r in rows3:
        ws3.append(r)

    ws4 = wb.create_sheet("inquiry-response")
    df_inq = pd.DataFrame(inquiry_responses)
    if df_inq.empty:
        df_inq = pd.DataFrame(columns=["email_id", "response"])
    df_inq = df_inq.rename(columns={"email_id": "email ID"})
    rows4 = dataframe_to_rows(df_inq[["email ID", "response"]], index=False, header=True)
    for r in rows4:
        ws4.append(r)

    ws5 = wb.create_sheet("updated-products")
    rows5 = dataframe_to_rows(updated_products, index=False, header=True)
    for r in rows5:
        ws5.append(r)

    wb.save("solution_output_ollama.xlsx")
    print("\nCreated 'solution_output_ollama.xlsx' with multiple sheets.")


# Main

In [ ]:
if __name__ == "__main__":
    if not verify_ollama():
        print("Connection failed. Ollama server not accessible.")
        exit(1)

    products_df, emails_df = load_data()

    classifier = EmailClassifier(model="llama3")
    emails_df = classifier.classify_all(emails_df)

    order_mgr = OrderManager(products_df, model="llama3")
    all_order_lines = []

    order_emails = emails_df[emails_df.category == "order_request"]
    for _, row in order_emails.iterrows():
        lines = order_mgr.process_order(row["message"], row["email_id"])
        all_order_lines.extend(lines)

    grouped_orders = {}
    for ln in all_order_lines:
        grouped_orders.setdefault(ln["email_id"], []).append(ln)
    order_responses = order_mgr.generate_order_response(grouped_orders)

    inq_mgr = ProductInquiryHandler(order_mgr.products, model="llama3")
    inquiry_res = []
    inquiry_emails = emails_df[emails_df.category == "product_inquiry"]
    for _, row in inquiry_emails.iterrows():
        ans = inq_mgr.handle_inquiry(row["message"])
        inquiry_res.append({"email_id": row["email_id"], "response": ans})

    generate_spreadsheet(emails_df, all_order_lines, order_responses, inquiry_res, order_mgr.products)
    print("Done. Check 'solution_output_ollama.xlsx'.")

Ollama server is accessible!
Data loaded successfully

Products Preview:


,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00



Emails Preview:


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."


  0%|          | 0/23 [00:00<?, ?it/s]

LLM extraction failed, falling back to regex extraction. Error: Empty products in extracted JSON.
Raw LLM output was: {"model":"llama3","created_at":"2025-04-09T16:27:26.9729566Z","response":"Based on the email, I can extract the product order information as follows:\n\n{\"products\":[{\"product_id\":\"LTH0976\",\"quantity\":1}]}\n\nNote: Since there is only one product mentioned in the email (LTH0976 Leather Bifold Wallets), and the quantity is not explicitly specified, I assumed a quantity of 1.","done":true,"done_reason":"stop","context":[128006,882,128007,271,30059,2027,10373,505,420,2613,627,5715,4823,304,279,3645,25,5324,10354,67682,3107,851,3332,55,2247,13832,794,17,2186,62499,633,4886,512,13347,1070,11,358,1390,311,2015,682,279,9861,445,3701,28384,21,32178,426,49385,37583,82,499,617,304,5708,13,358,2846,8736,709,264,2678,53185,8221,323,1521,1053,387,4832,369,856,15808,13,9930,499,4999,128009,128006,78191,128007,271,29815,389,279,2613,11,358,649,8819,279,2027,2015,2038,439,11263